<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vizro -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 8.1 MB/s eta 0:00:00


In [2]:
WEEK_DAYS = {
    1: "Monday",
    2: "Tuesday",
    3: "Wednesday",
    4: "Thursday",
    5: "Friday",
    6: "Saturday",
    7: "Sunday"
}

In [2]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8' )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [3]:
df_2 = select("SELECT *, date_part('h', ds) as hour, count - lag(count, 1) OVER (partition by platform, weekday, date_part('h', ds) order by ds) as wow_diff FROM vizro.yandex_data_agg")

In [4]:
df_2['date'] = df_2["ds"].dt.date

In [5]:
import vizro.models as vm
import vizro.plotly.express as px

from vizro import Vizro

heatmap_tabbed = vm.Tabs(
    tabs=[vm.Container(
            title="Weekly queries count",
            components=[
            vm.Graph(
            id="Weekly queries count",
            figure=px.density_heatmap(data_frame=df_2[df_2.date > df_2.date.max() - pd.Timedelta(7, 'days')], x="date", y="hour", z="count", histfunc="sum", text_auto=True, nbinsy=24, nbinsx=7, facet_col="platform")
            )]),
            vm.Container(
            title="WoW difference",
            components=[
            vm.Graph(
            id="WoW difference",
            figure=px.density_heatmap(df_2[df_2.date > df_2.date.max() - pd.Timedelta(7, 'days')], x="date", y="hour", z="wow_diff", histfunc="sum", text_auto=True, nbinsy=24, nbinsx=7, color_continuous_scale=px.colors.diverging.RdBu, color_continuous_midpoint=0, facet_col='platform')
            )])
        ])

page = vm.Page(title="My page", components=[heatmap_tabbed])
dashboard = vm.Dashboard(pages=[page])
Vizro().build(dashboard).run()

<IPython.core.display.Javascript object>